This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [3]:
!pip install python-dotenv

In [6]:
import os
from dotenv import load_dotenv

# Specifying the path to nasdaq.env
path_to_env = "/Users/arka_bagchi/Desktop/Springboard/dsc/Unit 7 - Data Wrangling/nasdaq.env"
load_dotenv(dotenv_path=path_to_env)

API_KEY = os.getenv('NASDAQ_API_KEY')
print(API_KEY)

_CV9WztniAoPyq5MNcrp


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [7]:
# First, import the relevant modules
import requests
from collections import defaultdict, namedtuple, Counter, deque
import json
import sys
import os
import math
import random
import pickle
import urllib

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [8]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# Define the endpoint URL
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"

# Define the parameters
params = {
    'start_date': '2021-01-01',
    'end_date': '2021-01-01',
    'api_key': API_KEY
}

# Make the API call
response = requests.get(url, params=params)

# Parse the JSON data
data = response.json()

# Print a glimpse of the data
print(json.dumps(data, indent=4))

{
    "dataset_data": {
        "limit": null,
        "transform": null,
        "column_index": null,
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Change",
            "Traded Volume",
            "Turnover",
            "Last Price of the Day",
            "Daily Traded Units",
            "Daily Turnover"
        ],
        "start_date": "2021-01-01",
        "end_date": "2020-12-01",
        "frequency": "daily",
        "data": [],
        "collapse": null,
        "order": null
    }
}


In [12]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
# Inspect top-level keys
print("Top-level keys:", data.keys())

Top-level keys: dict_keys(['dataset_data'])


In [13]:
# Inspect 'dataset_data' structure since it seems to be the main container for the data
print("\nKeys under 'dataset_data':", data['dataset_data'].keys())


Keys under 'dataset_data': dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [14]:
# Print the 'column_names' to understand the kind of data we're working with
print("\nColumn Names:", data['dataset_data']['column_names'])


Column Names: ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [16]:
def explore_structure(data, indent=0):
    """Recursively explore the structure of a dictionary."""
    
    # Check if the input is a dictionary
    if isinstance(data, dict):
        for key, value in data.items():
            print('  ' * indent + f'Key: {key}')
            explore_structure(value, indent+1)
    # Check if the input is a list
    elif isinstance(data, list):
        print('  ' * indent + f'List of length {len(data)}')
        # To avoid extensive printing, we can explore the structure of just the first item
        if data:
            explore_structure(data[0], indent+1)
    else:
        print('  ' * indent + f'Value: {data}')

# Use the function to explore the structure of our data
explore_structure(data)

Key: dataset_data
  Key: limit
    Value: None
  Key: transform
    Value: None
  Key: column_index
    Value: None
  Key: column_names
    List of length 11
      Value: Date
  Key: start_date
    Value: 2021-01-01
  Key: end_date
    Value: 2020-12-01
  Key: frequency
    Value: daily
  Key: data
    List of length 0
  Key: collapse
    Value: None
  Key: order
    Value: None


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [17]:
# Define the endpoint URL
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"

# Define the parameters: start and end date for the year 2017
params = {
    'start_date': '2017-01-01',
    'end_date': '2017-12-31',
    'api_key': API_KEY
}

# Make the API request
response = requests.get(url, params=params)

# Ensure the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data_2017 = response.json()
    print("Data successfully fetched!")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


Data successfully fetched!


In [20]:
# Extract the dataset from the JSON response
afx_2017_data = data_2017['dataset_data']

# Print the keys of the dictionary to see its structure
print(afx_2017_data.keys())

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [21]:
# Optionally, if we want to see the first few records of the data:
print(afx_2017_data['data'][:5])  # This will print the first 5 data records

[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None], ['2017-12-27', 51.45, 51.89, 50.76, 51.82, None, 57452.0, 2957018.0, None, None, None], ['2017-12-22', 51.05, 51.5, 50.92, 51.32, None, 71165.0, 3641949.0, None, None, None], ['2017-12-21', 51.16, 51.52, 50.9, 51.4, None, 120649.0, 6179433.0, None, None, None]]


In [22]:
# Extract the column names and find the index of the 'Open' prices
column_names = afx_2017_data['column_names']
open_index = column_names.index('Open')

# Extract the data and get all opening prices, excluding any missing values (None)
opening_prices = [daily_data[open_index] for daily_data in afx_2017_data['data'] if daily_data[open_index] is not None]

# Find the highest and lowest opening prices
highest_opening = max(opening_prices)
lowest_opening = min(opening_prices)

print(f"Highest opening price in 2017: {highest_opening}")
print(f"Lowest opening price in 2017: {lowest_opening}")


Highest opening price in 2017: 53.11
Lowest opening price in 2017: 34.0


In [23]:
# Identify the positions of the 'High' and 'Low' prices in the column names
high_index = column_names.index('High')
low_index = column_names.index('Low')

# Calculate the daily differences (High - Low) and find the maximum difference
daily_differences = [(daily_data[high_index] - daily_data[low_index]) for daily_data in afx_2017_data['data']]
largest_change = max(daily_differences)

print(f"Largest change in any one day in 2017: {largest_change:.2f}")

Largest change in any one day in 2017: 2.81


In [24]:
# Identify the position of the 'Close' price in the column names
close_index = column_names.index('Close')

# Extract closing prices from the data
closing_prices = [daily_data[close_index] for daily_data in afx_2017_data['data']]

# Calculate the absolute differences between consecutive days
daily_changes = [abs(closing_prices[i+1] - closing_prices[i]) for i in range(len(closing_prices)-1)]

# Get the maximum difference
largest_two_day_change = max(daily_changes)

print(f"Largest change between any two days based on Closing Price in 2017: {largest_two_day_change:.2f}")


Largest change between any two days based on Closing Price in 2017: 2.56


In [25]:
# Identify the position of the 'Traded Volume' in the column names
volume_index = column_names.index('Traded Volume')

# Extract trading volumes from the data
daily_volumes = [daily_data[volume_index] for daily_data in afx_2017_data['data']]

# Calculate the average trading volume
average_volume = sum(daily_volumes) / len(daily_volumes)

print(f"Average daily trading volume in 2017: {average_volume:.2f}")


Average daily trading volume in 2017: 89124.34


In [26]:
def median(lst):
    """Calculate the median of a list."""
    sorted_lst = sorted(lst)
    lst_len = len(sorted_lst)
    
    # Check if the list has an odd number of elements
    if lst_len % 2 == 1:
        return sorted_lst[lst_len // 2]
    else:
        left_mid = sorted_lst[(lst_len - 1) // 2]
        right_mid = sorted_lst[lst_len // 2]
        return (left_mid + right_mid) / 2

# Using the daily_volumes list we created in the previous task:
median_volume = median(daily_volumes)

print(f"Median trading volume in 2017: {median_volume:.2f}")

Median trading volume in 2017: 76286.00
